In [1]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate, HumanMessagePromptTemplate
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
import pandas as pd
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI 

from dotenv import load_dotenv
import pdb


In [2]:
!pip install chardet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: C:\Python312\python.exe -m pip install --upgrade pip


In [3]:
load_dotenv()

True

In [4]:
import chardet

ModuleNotFoundError: No module named 'chardet'

In [37]:
llm = ChatOpenAI(model="gpt-3.5-turbo",temperature=0)
embeddings = OpenAIEmbeddings(deployment="embeddings model", chunk_size=1) 

In [28]:
def load_pdf(pdf):
    ''' This utility is used to convert the pdfs into chunks of texts '''
    #loader = PyPDFLoader(pdf)
    loader = TextLoader(pdf,encoding='utf-8')
    documents = loader.load()

    text_splitter = CharacterTextSplitter(chunk_size = 500, chunk_overlap = 200)
    texts = text_splitter.split_documents(documents)
    return texts
    

In [ ]:
def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        rawdata = f.read()
    result = chardet.detect(rawdata)
    return result['encoding']

In [31]:
pdf_text = load_pdf("../Ethereum Foundation – Hands on Lab.docx")
doc_search_pdf = FAISS.from_documents(documents=pdf_text,embeddings=embeddings)
doc_search_pdf.save_local("./fiass_db/"+"Quantum Computing.pdf")

RuntimeError: Error loading ../Ethereum Foundation – Hands on Lab.docx

In [58]:
persisted_vectorstore = FAISS.load_local("./fiass_db/"+"Quantum Computing.pdf", embeddings)

    # Use RetrievalQA chain for orchestration
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
result = qa.run("What is Quantum Computing")
print(result)

ImportError: Could not import faiss python package. Please install it with `pip install faiss` or `pip install faiss-cpu` (depending on Python version).

In [39]:
def query_pdf(query):
    # Load document using PyPDFLoader document loader
    loader = PyPDFLoader("Downloads/Quantum Computing.pdf")
    documents = loader.load()
    # Split document in chunks
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
    docs = text_splitter.split_documents(documents=documents)

    embeddings = OpenAIEmbeddings()
    # Create vectors
    vectorstore = FAISS.from_documents(docs, embeddings)
    # Persist the vectors locally on disk
    vectorstore.save_local("faiss_index_constitution")

    # Load from local storage
    persisted_vectorstore = FAISS.load_local("faiss_index_constitution", embeddings)

    # Use RetrievalQA chain for orchestration
    qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=persisted_vectorstore.as_retriever())
    result = qa.run(query)
    print(result)



In [40]:

def main():
    query = input("Type in your query: \n")
    while query != "exit":
        query_pdf(query)
        query = input("Type in your query: \n")

In [41]:
main()

ImportError: pypdf package not found, please install it with `pip install pypdf`